## makemore: becoming a backprop ninja

In [36]:
# Downloaded from https://colab.research.google.com/drive/1WV2oi2fh9XXyldh02wupFQX0wh5ZC-z-?usp=sharing
# there no change change in the first several cells from last lecture

In [37]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
%matplotlib inline

In [38]:
# read in all the words
words = open('names.txt', 'r').read().splitlines()
print(len(words))
print(max(len(w) for w in words))
print(words[:8])

32033
15
['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']


In [39]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
vocab_size = len(itos)
print(itos)
print(vocab_size)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
27


In [40]:
# build the dataset
block_size = 3 # context length: how many characters do we take to predict the next one?

def build_dataset(words):
  X, Y = [], []

  for w in words:
    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      context = context[1:] + [ix] # crop and append

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr,  Ytr  = build_dataset(words[:n1])     # 80%
Xdev, Ydev = build_dataset(words[n1:n2])   # 10%
Xte,  Yte  = build_dataset(words[n2:])     # 10%

torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


In [41]:
# ok biolerplate done, now we get to the action:

In [42]:
# utility function we will use later when comparing manual gradients to PyTorch gradients
def cmp(s, dt, t):
  ex = torch.all(dt == t.grad).item()
  app = torch.allclose(dt, t.grad)
  maxdiff = (dt - t.grad).abs().max().item()
  print(f'{s:15s} | exact: {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}')

In [43]:
n_embd = 10 # the dimensionality of the character embedding vectors
n_hidden = 64 # the number of neurons in the hidden layer of the MLP

g = torch.Generator().manual_seed(2147483647) # for reproducibility
C  = torch.randn((vocab_size, n_embd),            generator=g)
# Layer 1
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * (5/3)/((n_embd * block_size)**0.5)
b1 = torch.randn(n_hidden,                        generator=g) * 0.1 # using b1 just for fun, it's useless because of BN
# Layer 2
W2 = torch.randn((n_hidden, vocab_size),          generator=g) * 0.1
b2 = torch.randn(vocab_size,                      generator=g) * 0.1
# BatchNorm parameters
bngain = torch.randn((1, n_hidden))*0.1 + 1.0
bnbias = torch.randn((1, n_hidden))*0.1

# Note: I am initializating many of these parameters in non-standard ways
# because sometimes initializating with e.g. all zeros could mask an incorrect
# implementation of the backward pass.

parameters = [C, W1, b1, W2, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters)) # number of parameters in total
for p in parameters:
  p.requires_grad = True

4137


In [44]:
batch_size = 32
n = batch_size # a shorter variable also, for convenience
# construct a minibatch
ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
Xb, Yb = Xtr[ix], Ytr[ix] # batch X,Y

In [45]:
# forward pass, "chunkated" into smaller steps that are possible to backward one at a time

emb = C[Xb] # embed the characters into vectors
embcat = emb.view(emb.shape[0], -1) # concatenate the vectors
# Linear layer 1
hprebn = embcat @ W1 + b1 # hidden layer pre-activation
# BatchNorm layer
bnmeani = 1/n*hprebn.sum(0, keepdim=True)
bndiff = hprebn - bnmeani
bndiff2 = bndiff**2
bnvar = 1/(n-1)*(bndiff2).sum(0, keepdim=True) # note: Bessel's correction (dividing by n-1, not n)
bnvar_inv = (bnvar + 1e-5)**-0.5
bnraw = bndiff * bnvar_inv
hpreact = bngain * bnraw + bnbias
# Non-linearity
h = torch.tanh(hpreact) # hidden layer
# Linear layer 2
logits = h @ W2 + b2 # output layer
# cross entropy loss (same as F.cross_entropy(logits, Yb))
logit_maxes = logits.max(1, keepdim=True).values
norm_logits = logits - logit_maxes # subtract max for numerical stability
counts = norm_logits.exp()
counts_sum = counts.sum(1, keepdims=True)
counts_sum_inv = counts_sum**-1 # if I use (1.0 / counts_sum) instead then I can't get backprop to be bit exact...
probs = counts * counts_sum_inv
logprobs = probs.log()
loss = -logprobs[range(n), Yb].mean()

# PyTorch backward pass
for p in parameters:
  p.grad = None
for t in [logprobs, probs, counts, counts_sum, counts_sum_inv, # afaik there is no cleaner way
          norm_logits, logit_maxes, logits, h, hpreact, bnraw,
         bnvar_inv, bnvar, bndiff2, bndiff, hprebn, bnmeani,
         embcat, emb]:
  t.retain_grad()
loss.backward()
loss

tensor(3.3212, grad_fn=<NegBackward0>)

In [46]:
# Exercise 1: backprop through the whole thing manually,
# backpropagating through exactly all of the variables
# as they are defined in the forward pass above, one by one

# -----------------
# YOUR CODE HERE :)
# -----------------

#dlogprobs = torch.tensor([-1/n] * n).float()
#logprobs.grad[0]
#-1.0/32.0
dlogprobs = F.one_hot(Yb, num_classes=vocab_size).float() * -1/n
cmp('logprobs', dlogprobs, logprobs)

dprobs = 1 / probs * dlogprobs
cmp('probs', dprobs, probs)

# Note that counts_sum_inv is 32x1, so dcounts_sum_inv should be that shape too.
# This is funky because dprobs is 32x27.
# Yet another weird thing is that counts and counts_sum_inv are intertwined, it's not possible to change one w/o changing the other...
# but I guess the chain rule lets us ignore that?
dcounts_sum_inv = (counts * dprobs).sum(1, keepdim=True)
cmp('counts_sum_inv', dcounts_sum_inv, counts_sum_inv)

dcounts_sum = -1 * (counts_sum ** -2) * dcounts_sum_inv
cmp('counts_sum', dcounts_sum, counts_sum)

# counts appears _twice_ in the computation graph, so we need to account for both occurrences.
dcounts = dcounts_sum.expand(*counts.shape) + (counts_sum_inv.expand(*probs.shape) * dprobs)
cmp('counts', dcounts, counts)

dnorm_logits = norm_logits.exp() * dcounts
cmp('norm_logits', dnorm_logits, norm_logits)

dlogit_maxes = (-torch.ones((n, 1)) * dnorm_logits).sum(1, keepdim=True)
cmp('logit_maxes', dlogit_maxes, logit_maxes)

# logits also appears twice in the computation graph.
dlogits = F.one_hot(logits.argmax(1), num_classes=vocab_size).float() * dlogit_maxes + torch.ones((n, vocab_size)) * dnorm_logits
cmp('logits', dlogits, logits)


logprobs        | exact: True  | approximate: True  | maxdiff: 0.0
probs           | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum_inv  | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum      | exact: True  | approximate: True  | maxdiff: 0.0
counts          | exact: True  | approximate: True  | maxdiff: 0.0
norm_logits     | exact: True  | approximate: True  | maxdiff: 0.0
logit_maxes     | exact: True  | approximate: True  | maxdiff: 0.0
logits          | exact: True  | approximate: True  | maxdiff: 0.0


<!-- ![derivation of dh](derive_dh.jpg) -->
Deriving `dh` was not easy...

<img src="derive_dh.jpg" height="600">

In [47]:
# See above for the derivation.
dh = dlogits @ W2.T
cmp('h', dh, h)

h               | exact: True  | approximate: True  | maxdiff: 0.0


Deriving `dW2` was a bit easier.

<img src="derive_dW2.jpg" height="300">

In [48]:
# See above for derivation.
dW2 = h.T @ dlogits
cmp('W2', dW2, W2)

W2              | exact: True  | approximate: True  | maxdiff: 0.0


Um... I wrote this down in pencil and gave it to Claude and it converted it to LaTeX. It didn't get the `+`'s but that was easy to fix.

$$
\frac{dL}{db} = \begin{bmatrix} 
\frac{dL}{dY_{11}} + \frac{dL}{dY_{21}} & \frac{dL}{dY_{12}} + \frac{dL}{dY_{22}} 
\end{bmatrix}
$$

In [67]:
# See above for derivation.
db2 = dlogits.sum(0, keepdim=False)
cmp('b2', db2, b2)

dhpreact = torch.cosh(hpreact)**-2 * dh
cmp('hpreact', dhpreact, hpreact)

dbngain = (bnraw * dhpreact).sum(0, keepdim=True)

cmp('bngain', dbngain, bngain)

dbnbias = dhpreact.sum(0, keepdim=True)

cmp('bnbias', dbnbias, bnbias)

dbnraw = bngain * dhpreact

cmp('bnraw', dbnraw, bnraw)

dbnvar_inv = (bndiff * dbnraw).sum(0, keepdim=True)

cmp('bnvar_inv', dbnvar_inv, bnvar_inv)

dbnvar = (-0.5 * ((bnvar + 1e-5) ** -1.5)) * dbnvar_inv

cmp('bnvar', dbnvar, bnvar)

# bnvar = 1x64
# bndiff2 = 32x64
# bnvar[0] = [1/(n-1) * (bndiff2[0][0] + bndiff2[1][0] + ... + bndiff2[31][0])]
# dbndiff2[0][0] = 1/(n-1) * bndiff2[0][0] * dbnvar[0]
# dbndiff2[1][0] = 1/(n-1) * bndiff2[1][0] * dbnvar[0]
# ...
# dbndiff2[31][0] = 1/(n-1) * bndiff2[31][0] * dbnvar[0]

dbndiff2 = 1/(n-1) * dbnvar.expand((n, n_hidden))
cmp('bndiff2', dbndiff2, bndiff2)

# bndiff is used twice in the computation graph.
dbndiff = 2 * bndiff * dbndiff2 + bnvar_inv * dbnraw

cmp('bndiff', dbndiff, bndiff)

dbnmeani = -dbndiff.sum(0, keepdim=True)

cmp('bnmeani', dbnmeani, bnmeani)

# hprebn is used twice in the computation graph.
dhprebn = 1/n * dbnmeani + dbndiff

cmp('hprebn', dhprebn, hprebn)

# Here we're just using the same pattern we derived from backprop'ing the hidden layer.

dembcat = dhprebn @ W1.T
cmp('embcat', dembcat, embcat)

dW1 = embcat.T @ dhprebn
cmp('W1', dW1, W1)

db1 = dhprebn.sum(0, keepdim=True)
cmp('b1', db1, b1)

b2              | exact: True  | approximate: True  | maxdiff: 0.0
hpreact         | exact: False | approximate: True  | maxdiff: 9.313225746154785e-10
bngain          | exact: False | approximate: True  | maxdiff: 3.725290298461914e-09
bnbias          | exact: False | approximate: True  | maxdiff: 3.725290298461914e-09
bnraw           | exact: False | approximate: True  | maxdiff: 1.862645149230957e-09
bnvar_inv       | exact: False | approximate: True  | maxdiff: 3.725290298461914e-09
bnvar           | exact: False | approximate: True  | maxdiff: 9.313225746154785e-10
bndiff2         | exact: False | approximate: True  | maxdiff: 2.9103830456733704e-11
bndiff          | exact: False | approximate: True  | maxdiff: 9.313225746154785e-10
bnmeani         | exact: False | approximate: True  | maxdiff: 3.725290298461914e-09
hprebn          | exact: False | approximate: True  | maxdiff: 9.313225746154785e-10
embcat          | exact: False | approximate: True  | maxdiff: 2.7939677238464355e

In [137]:
# There's no math here; embcat is just a view on emb, so we'll just reverse the view.
demb = dembcat.view((emb.shape))

cmp('emb', demb, emb)

(C.shape, demb.shape, C[0])

def compute_dC():
    dC = torch.zeros(C.shape)
    for d_example, example_indexes in zip(demb, Xb):
        dC.index_add_(0, example_indexes, d_example)
    return dC

dC = compute_dC()

cmp('C', dC, C)

# (demb.shape, Xb.shape, C.shape)

#boop = torch.range(0, 8).view((3,3))

#boop.index_add_(0, torch.tensor([1, 1]), torch.tensor([[0.5, 0.5, 0.5], [0.5, 0.5, 0.5]]))

#boop


emb             | exact: False | approximate: True  | maxdiff: 2.7939677238464355e-09
C               | exact: False | approximate: True  | maxdiff: 1.30385160446167e-08
